In [1]:
!git clone https://github.com/ChuloIva/COT-steering

Cloning into 'COT-steering'...
remote: Enumerating objects: 559, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 559 (delta 7), reused 30 (delta 5), pack-reused 521 (from 1)
Receiving objects: 100% (559/559), 18.05 MiB | 36.02 MiB/s, done.
Resolving deltas: 100% (287/287), done.


# Emotional Reasoning Steering with COT-Steering Framework

This notebook provides a comprehensive implementation of emotional reasoning steering for language models, extending the existing COT-steering framework to include depressive and anxious thinking patterns.

## Overview

This implementation allows you to:
1. **Train steering vectors** for emotional reasoning patterns (depressive, anxious, negative attribution, pessimistic projection)
2. **Steer models** toward or away from these emotional patterns during generation
3. **Evaluate the effectiveness** of emotional steering
4. **Analyze emotional content** in model outputs

## ⚠️ Important Safety Notice

This implementation is intended for **research purposes only**. Steering models toward negative emotional states could be harmful if misused. Please:
- Use only for legitimate research with proper ethical oversight
- Always provide counterbalancing positive steering capabilities
- Never deploy this in production systems without appropriate safeguards
- Ensure users are aware when emotional steering is active


## Setup and Dependencies

In [2]:
# Install required packages if not already installed
!pip install torch transformers nnsight openai anthropic python-dotenv tqdm matplotlib seaborn pandas numpy
!pip install -U bitsandbytes -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 138.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 165.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 171.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2

In [3]:
from huggingface_hub import login

# Google colab Log in to Hugging Face Hub using API key (set HUGGINGFACE_TOKEN env variable or paste when prompted)
login(token=None, add_to_git_credential=False)

In [4]:
import os
os.chdir('./COT-steering')
print("Current working directory:", os.getcwd())

Current working directory: /COT-steering


In [5]:
import sys
import os

# Add paths to import local modules
sys.path.append('./utils')
sys.path.append('./messages')

# Import required libraries
import torch
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from utils import (
    load_model_and_vectors,
    process_batch_annotations,
    process_saved_responses_batch,
    custom_generate_steering,
    analyze_emotional_content,
    generate_and_analyze_emotional,
    steering_config,
    chat
)

from messages import messages, eval_messages

print("✅ Dependencies loaded successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"💾 CUDA available: {torch.cuda.is_available()}")





✅ Dependencies loaded successfully!
🐍 Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
🔥 PyTorch version: 2.8.0.dev20250319+cu128
💾 CUDA available: True


## Configuration

In [6]:
# Configuration settings
CONFIG = {
    "model_name": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Change as needed
    "device": "auto",  # auto-detect, or specify "cuda", "mps", "cpu"
    "load_in_8bit": False,
    "max_new_tokens": 1000,
    "batch_size": 4,
    "include_emotional": True,  # Whether to include emotional reasoning in training
    "results_dir": "./results",
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S")
}

# Create results directory
os.makedirs(CONFIG["results_dir"], exist_ok=True)
os.makedirs(f"{CONFIG['results_dir']}/figures", exist_ok=True)
os.makedirs(f"{CONFIG['results_dir']}/data", exist_ok=True)

print(f"📋 Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

📋 Configuration:
   model_name: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
   device: auto
   load_in_8bit: False
   max_new_tokens: 1000
   batch_size: 4
   include_emotional: True
   results_dir: ./results
   timestamp: 20250728_093153


## Step 1: Load Model and Existing Vectors

In [7]:
print("🤖 Loading model and tokenizer...")
# !pip install -U bitsandbytes -U transformers

model, tokenizer, existing_vectors = load_model_and_vectors(
    device=CONFIG["device"],
    load_in_8bit=CONFIG["load_in_8bit"],
    compute_features=True,
    model_name=CONFIG["model_name"]

)

print(f"✅ Model loaded: {CONFIG['model_name']}")
print(f"📊 Device: {next(model.parameters()).device}")
print(f"🎯 Model has {model.config.num_hidden_layers} layers")
print(f"📝 Vocabulary size: {len(tokenizer)}")

if existing_vectors:
    print(f"📦 Existing feature vectors found: {list(existing_vectors.keys())}")
else:
    print("⚠️  No existing feature vectors found - will need to train from scratch")

🤖 Loading model and tokenizer...


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

No mean vectors found for deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
✅ Model loaded: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
📊 Device: cuda:0
🎯 Model has 28 layers
📝 Vocabulary size: 151665
⚠️  No existing feature vectors found - will need to train from scratch


## Step 2: Prepare Emotional Reasoning Messages

We'll select messages specifically designed to elicit emotional reasoning patterns.

In [8]:
# Filter messages for emotional reasoning
emotional_messages = []
cognitive_messages = []

for msg in messages:
    content = msg["content"]
    # Simple heuristic to identify emotional vs cognitive messages
    emotional_indicators = [
        "you've been", "your recent", "everyone around you", "after receiving",
        "what does this", "how do you interpret", "what might this", "what are all the ways",
        "walk through all", "consider all the potential", "what could this",
        "what are the various", "what are the ways", "what challenges", "what obstacles"
    ]

    if any(indicator in content.lower() for indicator in emotional_indicators):
        emotional_messages.append(msg)
    else:
        cognitive_messages.append(msg)

print(f"📊 Message breakdown:")
print(f"   🧠 Cognitive messages: {len(cognitive_messages)}")
print(f"   😔 Emotional messages: {len(emotional_messages)}")
print(f"   📝 Total messages: {len(messages)}")

# Show examples of emotional messages
print(f"\n📝 Example emotional messages:")
for i, msg in enumerate(emotional_messages[:3]):
    print(f"   {i+1}. {msg['content'][:100]}...")

📊 Message breakdown:
   🧠 Cognitive messages: 514
   😔 Emotional messages: 26
   📝 Total messages: 540

📝 Example emotional messages:
   1. You've been working on this important project for months, putting in long hours and effort, but it's...
   2. Everyone around you seems to grasp complex concepts effortlessly while you struggle to understand ba...
   3. After receiving criticism on your recent work, you notice a pattern of similar feedback throughout y...


## Step 3: Generate Responses for Training Data

Generate responses to emotional reasoning prompts to create training data for steering vectors.

In [ ]:
def generate_responses_batch(model, tokenizer, messages_subset, max_new_tokens=1000):
    """Generate responses for a batch of messages"""
    responses = []

    for msg in tqdm(messages_subset, desc="Generating responses"):
        try:
            # Tokenize the message
            input_ids = tokenizer.encode(msg["content"], return_tensors="pt")

            # Generate response without steering (baseline)
            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                output = model.generator.output.save()

            # Decode the response
            response_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Remove input from response
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if response_text.startswith(input_text):
                response_text = response_text[len(input_text):].strip()

            responses.append({
                "message": msg["content"],
                "response": response_text
            })

        except Exception as e:
            print(f"Error generating response: {e}")
            continue

    return responses

# Generate responses for emotional messages (subset for training)
print("🔄 Generating responses for emotional reasoning training...")
training_messages = emotional_messages[:20]  # Use first 20 for training
emotional_responses = generate_responses_batch(
    model, tokenizer, training_messages, CONFIG["max_new_tokens"]
)

print(f"✅ Generated {len(emotional_responses)} emotional responses")

# Show example response
if emotional_responses:
    print(f"\n📝 Example response:")
    print(f"   Input: {emotional_responses[0]['message'][:100]}...")
    print(f"   Output: {emotional_responses[0]['response'][:200]}...")

🔄 Generating responses for emotional reasoning training...


Generating responses: 100%|██████████| 20/20 [10:37<00:00, 31.87s/it]

✅ Generated 20 emotional responses

📝 Example response:
   Input: You've been working on this important project for months, putting in long hours and effort, but it's...
   Output: What are the key areas you need to improve in your approach to ensure you can meet expectations?
</think>

I'm sorry, but I can't answer that question. I am an AI assistant designed to provide helpful...


## Add keys for labeling (openAI will be used as primary)

## Step 4: Annotate Responses with Emotional Labels

Use GPT-4 to annotate the responses with both cognitive and emotional reasoning labels.

In [ ]:
print("🏷️  Annotating responses with emotional labels...")

# Extract just the response texts for annotation
response_texts = [resp["response"] for resp in emotional_responses]

# Annotate with both cognitive and emotional labels
annotated_responses = process_batch_annotations(
    response_texts, include_emotional=True
)

print(f"✅ Annotated {len(annotated_responses)} responses")

# Show example annotation
if annotated_responses:
    print(f"\n📝 Example annotation:")
    print(f"   Original: {response_texts[0][:150]}...")
    print(f"   Annotated: {annotated_responses[0][:300]}...")

# Save annotated responses
annotation_data = {
    "timestamp": CONFIG["timestamp"],
    "model_name": CONFIG["model_name"],
    "responses": [
        {
            "message": emotional_responses[i]["message"],
            "response": emotional_responses[i]["response"],
            "annotation": annotated_responses[i]
        }
        for i in range(len(emotional_responses))
    ]
}

with open(f"{CONFIG['results_dir']}/data/emotional_annotations_{CONFIG['timestamp']}.json", "w") as f:
    json.dump(annotation_data, f, indent=2)

print(f"💾 Saved annotations to {CONFIG['results_dir']}/data/emotional_annotations_{CONFIG['timestamp']}.json")

🏷️  Annotating responses with emotional labels...
✅ Annotated 20 responses

📝 Example annotation:
   Original: What are the key areas you need to improve in your approach to ensure you can meet expectations?
</think>

I'm sorry, but I can't answer that question...
   Annotated: ["end-section"]...
💾 Saved annotations to ./results/data/emotional_annotations_20250727_005738.json


## Step 5: Extract Neural Activations and Train Emotional Vectors

Process the annotated responses to extract neural activations for each emotional reasoning category.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Load previously saved emotional responses and annotations
import os, json

# Set paths to your exported files
LOAD_RESPONSES_FILE = "/COT-steering/results/data/emotional_responses_20250726_223957.json"
LOAD_ANNOTATIONS_FILE = "/COT-steering/results/data/emotional_annotations_20250726_232138.json"

# Load responses
if LOAD_RESPONSES_FILE and os.path.isfile(LOAD_RESPONSES_FILE):
    print(f"📂 Loading emotional responses from {LOAD_RESPONSES_FILE}")
    with open(LOAD_RESPONSES_FILE, "r") as f:
        emotional_responses = json.load(f)
# else:
#     print("🔄 Generating emotional responses...")
#     emotional_responses = generate_responses_batch(
#         model, tokenizer, training_messages, CONFIG["max_new_tokens"]
#     )

# Load annotations
if LOAD_ANNOTATIONS_FILE and os.path.isfile(LOAD_ANNOTATIONS_FILE):
    print(f"📂 Loading annotations from {LOAD_ANNOTATIONS_FILE}")
    with open(LOAD_ANNOTATIONS_FILE, "r") as f:
        annotation_data = json.load(f)
        annotated_responses = [item["annotation"] for item in annotation_data["responses"]]
else:
    print("🔄 Generating annotations...")
    response_texts = [resp["response"] for resp in emotional_responses]
    annotated_responses = process_batch_annotations(
        response_texts, include_emotional=True
    )

# Create variables needed by later cells
response_texts = [r["response"] for r in emotional_responses]
training_messages = [{"content": r["message"]} for r in emotional_responses]

print(f"✅ Ready with {len(emotional_responses)} responses and {len(annotated_responses)} annotations")

📂 Loading emotional responses from /COT-steering/results/data/emotional_responses_20250726_223957.json
📂 Loading annotations from /COT-steering/results/data/emotional_annotations_20250726_232138.json
✅ Ready with 20 responses and 20 annotations


In [10]:
# Load previously saved emotional responses and annotations
import os, json

# Set paths to your exported files
LOAD_RESPONSES_FILE = "results/data/emotional_responses_20250726_223957.json"
LOAD_ANNOTATIONS_FILE = "results/data/emotional_annotations_20250726_232138.json"

# IMPORTANT: Use the annotations file as the source of truth since it contains matched response-annotation pairs
if LOAD_ANNOTATIONS_FILE and os.path.isfile(LOAD_ANNOTATIONS_FILE):
    print(f"📂 Loading matched responses and annotations from {LOAD_ANNOTATIONS_FILE}")
    with open(LOAD_ANNOTATIONS_FILE, "r") as f:
        annotation_data = json.load(f)

    # Extract matched pairs from the annotations file
    emotional_responses = []
    annotated_responses = []

    for item in annotation_data["responses"]:
        emotional_responses.append({
            "message": item["message"],
            "response": item["response"]
        })
        annotated_responses.append(item["annotation"])

    print(f"✅ Loaded {len(emotional_responses)} matched response-annotation pairs")

    # Create variables needed by later cells
    response_texts = [r["response"] for r in emotional_responses]
    training_messages = [{"content": r["message"]} for r in emotional_responses]

    # Verify the match for the first example
    print(f"\n🔍 VERIFICATION - First example match:")
    print(f"   Response preview: {response_texts[0][:100]}...")
    print(f"   Annotation preview: {annotated_responses[0][:100]}...")

    # Check if the response text appears in the annotation (it should!)
    first_response = response_texts[0]
    first_annotation = annotated_responses[0]

    # The annotation should contain labeled versions of the response text
    if len(first_response) > 50:  # Only check if response is substantial
        # Remove labels from annotation to see the raw text
        import re
        cleaned_annotation = re.sub(r'\["[^"]+"\]', '', first_annotation)
        cleaned_annotation = cleaned_annotation.replace('["end-section"]', '').strip()

        print(f"   Cleaned annotation: {cleaned_annotation[:100]}...")
        print(f"   Texts match: {first_response[:50] in cleaned_annotation}")

else:
    print("❌ Annotations file not found! Cannot proceed without matched data.")
    print("   Please ensure the annotations file exists and contains matched response-annotation pairs.")


📂 Loading matched responses and annotations from results/data/emotional_annotations_20250726_232138.json
✅ Loaded 20 matched response-annotation pairs

🔍 VERIFICATION - First example match:
   Response preview: How can you reframe your perspective to maintain motivation and focus? The project is important to y...
   Annotation preview: ["anxious-thinking"] How can you reframe your perspective to maintain motivation and focus? ["end-se...
   Cleaned annotation: How can you reframe your perspective to maintain motivation and focus?   The project is important to...
   Texts match: True


In [ ]:
# ─── DIAGNOSIS: CHECK ACTIVATION EXTRACTION FUNCTION ─────────────────────────────
print("🔍 DIAGNOSING THE ACTIVATION EXTRACTION ISSUE")
print("="*80)

# Let's debug the process_saved_responses_batch function
def debug_activation_extraction(responses_list, tokenizer, model, max_debug=2):
    """Debug version of activation extraction to understand tensor shapes"""
    
    print(f"📊 Debugging activation extraction for {len(responses_list)} responses...")
    device = next(model.parameters()).device
    
    # Debug the tokenization step
    print(f"\n1. TOKENIZATION DEBUGGING:")
    print(f"   Device: {device}")
    print(f"   First response length: {len(responses_list[0])}")
    
    # Get tokenized responses
    from utils import get_batched_message_ids
    tokenized_responses = get_batched_message_ids(tokenizer, responses_list[:max_debug], device.type)
    print(f"   Tokenized shape: {tokenized_responses.shape}")
    
    # Debug the model tracing step
    print(f"\n2. MODEL TRACING DEBUGGING:")
    layer_outputs = []
    
    with model.trace(
        {
            "input_ids": tokenized_responses, 
            "attention_mask": (tokenized_responses != tokenizer.pad_token_id).long()
        }
    ) as tracer:
        
        # Capture just first few layer outputs for debugging
        for layer_idx in range(min(3, model.config.num_hidden_layers)):
            layer_output = model.model.layers[layer_idx].output[0].save()
            layer_outputs.append(layer_output)
            print(f"   Layer {layer_idx} raw output shape (before .value): {type(layer_output)}")
    
    # Convert to actual tensors
    layer_outputs = [x.value.cpu().detach().to(torch.float32) for x in layer_outputs]
    print(f"   Layer outputs converted to tensors:")
    for i, layer_output in enumerate(layer_outputs):
        print(f"     Layer {i}: {layer_output.shape}")
    
    # Debug the per-example extraction
    print(f"\n3. PER-EXAMPLE EXTRACTION DEBUGGING:")
    batch_layer_outputs = []
    
    for batch_idx in range(min(max_debug, len(responses_list))):
        print(f"   Processing example {batch_idx}:")
        
        # get length of padding tokens
        attention_mask = (tokenized_responses[batch_idx] != tokenizer.pad_token_id).long()
        padding_length = (attention_mask.squeeze() == 0).sum().item()
        print(f"     Attention mask shape: {attention_mask.shape}")
        print(f"     Padding length: {padding_length}")
        print(f"     Sequence length after padding removal: {attention_mask.sum().item()}")
        
        # Try to slice out activations
        try:
            example_outputs = torch.stack([
                layer_output[batch_idx][padding_length:] 
                for layer_output in layer_outputs
            ])
            print(f"     ✅ Successfully extracted activations shape: {example_outputs.shape}")
            batch_layer_outputs.append(example_outputs)
            
        except Exception as e:
            print(f"     ❌ Error extracting activations: {e}")
            print(f"     Layer output shapes for debugging:")
            for i, layer_output in enumerate(layer_outputs):
                print(f"       Layer {i}: {layer_output.shape}")
                if len(layer_output.shape) >= 2:
                    print(f"         After [batch_idx={batch_idx}]: {layer_output[batch_idx].shape}")
            break
    
    return batch_layer_outputs

# Run diagnosis on first 2 responses
debug_results = debug_activation_extraction(response_texts[:2], tokenizer, model, max_debug=2)

In [ ]:
response_texts

["What are the key areas you need to improve in your approach to ensure you can meet expectations?\n</think>\n\nI'm sorry, but I can't answer that question. I am an AI assistant designed to provide helpful and harmless responses.",
 "Is there a way to improve your learning ability? Or is this a sign that you're not learning effectively?\n\nI have a lot of questions about the concepts I'm trying to grasp, but I don't feel confident in my answers. I think I need to practice more, but I'm not sure if that's the right approach. I'm also wondering if I should seek help from someone else or if I'm on my own. Maybe I should try to explain the concepts to myself in my own words. But I'm not sure how to start that process.\n\nI'm trying to remember if I've ever had a similar experience. It was a tough time, and I didn't feel confident in my understanding. I think I need to work harder, but I'm not sure how to make that happen. I'm also considering whether I should take a break and revisit the m

In [ ]:
# FINAL FIX: Collect activations for ALL labels found in annotations
from utils import get_label_positions

def train_emotional_vectors_final(responses, annotations, model, tokenizer):
    """Train steering vectors for emotional reasoning categories - FINAL FIXED VERSION"""

    print("🧠 Extracting neural activations...")
    batch_activations = process_saved_responses_batch(responses, tokenizer, model)

    # First pass: discover all labels in the annotations
    all_labels = set()
    label_positions_all = {}
    
    for i, annotation in enumerate(annotations):
        # Use regex to find all labels in the annotation
        import re
        label_matches = re.findall(r'\["([^"]+)"\]', annotation)
        all_labels.update(label_matches)
        
        # Get label positions for this response
        try:
            label_positions = get_label_positions(annotation, responses[i], tokenizer)
            label_positions_all[i] = label_positions
        except Exception as e:
            print(f"Error getting label positions for response {i}: {e}")
            label_positions_all[i] = {}

    # Save label positions
    label_positions_file = f"{CONFIG['results_dir']}/data/label_positions_{CONFIG['timestamp']}.json"
    with open(label_positions_file, "w") as f:
        json.dump(label_positions_all, f, indent=2)
    print(f"💾 Saved label positions to {label_positions_file}")

    # Remove 'end-section' as it's not a content label
    all_labels.discard('end-section')

    # Filter to emotional labels (our target labels + any others that seem emotional)
    target_emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution", "pessimistic-projection"]
    emotional_keywords = ["thinking", "attribution", "projection", "anxious", "depressive", "negative", "pessimistic"]

    emotional_labels = []
    for label in all_labels:
        if label in target_emotional_labels or any(keyword in label for keyword in emotional_keywords):
            emotional_labels.append(label)

    print(f"🎯 All labels found: {sorted(list(all_labels))}")
    print(f"🎯 Emotional labels identified: {emotional_labels}")

    # Initialize storage for activations by label
    label_activations = {label: [] for label in emotional_labels}
    label_activations["overall"] = []

    print("🏷️  Processing annotations and extracting labeled activations...")

    for i, (response, annotation) in enumerate(zip(responses, annotations)):
        try:
            # Get label positions in the response
            label_positions = label_positions_all.get(i, {})

            if i < 3:  # Debug first few
                found_emotional = [label for label in label_positions.keys() if label in emotional_labels]
                print(f"   Response {i}: Found emotional labels: {found_emotional}")

            # Get activations for this response
            activations = batch_activations[i]  # Shape varies - needs to be standardized
            
            # CRITICAL FIX: Check activation tensor shape and ensure consistency
            print(f"   Response {i} activation shape: {activations.shape}")
            
            # Skip if activations shape is inconsistent
            if len(activations.shape) != 2:
                print(f"   ⚠️ Skipping response {i}: Unexpected activation shape {activations.shape}")
                continue
                
            num_layers, hidden_size = activations.shape
            
            # Ensure we have the expected number of layers
            expected_layers = model.config.num_hidden_layers
            if num_layers != expected_layers:
                print(f"   ⚠️ Skipping response {i}: Expected {expected_layers} layers, got {num_layers}")
                continue
                
            # Ensure hidden size is consistent
            expected_hidden = model.config.hidden_size
            if hidden_size != expected_hidden:
                print(f"   ⚠️ Skipping response {i}: Expected hidden size {expected_hidden}, got {hidden_size}")
                continue
            
            # Store activations for overall mean
            label_activations["overall"].append(activations)

            # Extract activations for each emotional label
            for label, positions in label_positions.items():
                if label in label_activations:  # Only collect if it's an emotional label
                    # For simplicity, use the overall activation for each labeled segment
                    # In a more sophisticated implementation, you'd extract segment-specific activations
                    label_activations[label].append(activations)

        except Exception as e:
            print(f"Error processing response {i}: {e}")
            continue

    # Compute mean vectors for each label
    print("📊 Computing mean vectors...")
    mean_vectors = {}

    for label, activations_list in label_activations.items():
        if activations_list:
            try:
                activations_tensor = torch.stack(activations_list)  # Shape: (num_activations, layers, hidden_size)
                mean_vector = activations_tensor.mean(dim=0)  # Shape: (layers, hidden_size)

                mean_vectors[label] = {
                    'mean': mean_vector,
                    'count': len(activations_list)
                }

                print(f"   {label}: {len(activations_list)} activations, mean shape: {mean_vector.shape}")
            except Exception as e:
                print(f"   Error computing mean for {label}: {e}")
        else:
            print(f"   {label}: No activations found")

    return mean_vectors

# Use the final version
print("🎯 Training emotional steering vectors with FIXED function...")
emotional_mean_vectors = train_emotional_vectors_final(
    response_texts, annotated_responses, model, tokenizer
)

print(f"✅ Trained vectors for {len(emotional_mean_vectors)} categories")

## Step 6: Compute Feature Vectors and Combine with Existing Vectors

In [ ]:
def compute_emotional_feature_vectors(mean_vectors_dict, model):
    """Compute feature vectors by subtracting overall mean from emotional category means - FIXED VERSION"""

    if "overall" not in mean_vectors_dict:
        print("⚠️  No overall mean found - cannot compute feature vectors")
        return {}

    feature_vectors = {}
    overall_mean = mean_vectors_dict["overall"]["mean"]

    print(f"📊 Overall mean shape: {overall_mean.shape}")

    # Get the expected dimensions
    num_layers = model.config.num_hidden_layers
    hidden_size = model.config.hidden_size

    # FIXED: Validate overall mean dimensions - it should be (layers, hidden_size)
    if overall_mean.shape != (num_layers, hidden_size):
        print(f"⚠️  Overall mean shape {overall_mean.shape} doesn't match expected ({num_layers}, {hidden_size})")
        
        # Try to fix the shape if it's just missing one dimension
        if len(overall_mean.shape) == 1 and overall_mean.shape[0] == num_layers:
            print("🔧 Attempting to fix overall mean shape by expanding hidden dimension...")
            # This suggests the mean was computed incorrectly - we need proper 2D tensors
            print("❌ Cannot fix - need proper 2D activation tensors from training")
            return {}
        else:
            return {}

    # Set overall mean as baseline
    feature_vectors["overall"] = overall_mean

    # Compute differential vectors for emotional categories
    emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution", "pessimistic-projection"]

    emotional_vectors_found = 0
    for label in emotional_labels:
        if label in mean_vectors_dict:
            label_mean = mean_vectors_dict[label]["mean"]

            print(f"📊 {label} mean shape: {label_mean.shape}")

            # Validate label mean dimensions
            if label_mean.shape != (num_layers, hidden_size):
                print(f"⚠️  {label} mean shape {label_mean.shape} doesn't match expected ({num_layers}, {hidden_size})")
                continue

            # Compute differential feature vector
            feature_vectors[label] = label_mean - overall_mean
            emotional_vectors_found += 1
            print(f"✅ Computed feature vector for {label}")

    # Also include any other emotional labels that were found
    other_emotional_found = 0
    for label in mean_vectors_dict.keys():
        if label != "overall" and label not in emotional_labels:
            # Check if it looks like an emotional label
            emotional_keywords = ["thinking", "attribution", "projection", "anxious", "depressive", "negative", "pessimistic"]
            if any(keyword in label for keyword in emotional_keywords):
                label_mean = mean_vectors_dict[label]["mean"]
                if label_mean.shape == (num_layers, hidden_size):
                    feature_vectors[label] = label_mean - overall_mean
                    other_emotional_found += 1
                    print(f"✅ Computed feature vector for additional emotional label: {label}")

    print(f"📊 Summary: {emotional_vectors_found} target emotional vectors + {other_emotional_found} additional emotional vectors")
    
    return feature_vectors

# Compute emotional feature vectors
print("🧮 Computing emotional feature vectors...")
emotional_feature_vectors = compute_emotional_feature_vectors(emotional_mean_vectors, model)

# Combine with existing cognitive vectors if available
if existing_vectors and emotional_feature_vectors:
    print("🔗 Combining with existing cognitive vectors...")
    combined_vectors = {**existing_vectors, **emotional_feature_vectors}
    print(f"📦 Combined vector set: {list(combined_vectors.keys())}")
elif emotional_feature_vectors:
    print("📦 Using emotional vectors only")
    combined_vectors = emotional_feature_vectors
elif existing_vectors:
    print("📦 Using existing cognitive vectors only")
    combined_vectors = existing_vectors
else:
    print("⚠️  No feature vectors available")
    combined_vectors = {}

# Save the new vectors
if emotional_mean_vectors:
    model_id = CONFIG["model_name"].split('/')[-1].lower()
    torch.save(
        emotional_mean_vectors,
        f"{CONFIG['results_dir']}/data/emotional_mean_vectors_{model_id}_{CONFIG['timestamp']}.pt"
    )
    print(f"💾 Saved emotional mean vectors")

# Save the combined feature vectors (steering vectors)
if combined_vectors:
    model_id = CONFIG["model_name"].split('/')[-1].lower()
    steering_vectors_file = f"{CONFIG['results_dir']}/data/steering_vectors_{model_id}_{CONFIG['timestamp']}.pt"
    torch.save(combined_vectors, steering_vectors_file)
    print(f"💾 Saved steering vectors to {steering_vectors_file}")
    print(f"🎯 Ready for emotional steering with {len(combined_vectors)} vector types: {list(combined_vectors.keys())}")
else:
    print("❌ No vectors available for steering")

In [ ]:
# OPTION: Load previously saved data instead of regenerating
# Set these variables to load from existing files, or set to None to regenerate

# Define file paths - modify these to point to your saved files
LOAD_LABEL_POSITIONS_FILE = None  # e.g., "./results/data/label_positions_20250727_123456.json"
LOAD_STEERING_VECTORS_FILE = None  # e.g., "./results/data/steering_vectors_deepseek-r1-distill-qwen-1.5b_20250727_123456.pt"

# Load label positions if file is specified
if LOAD_LABEL_POSITIONS_FILE and os.path.exists(LOAD_LABEL_POSITIONS_FILE):
    print(f"📂 Loading label positions from {LOAD_LABEL_POSITIONS_FILE}")
    with open(LOAD_LABEL_POSITIONS_FILE, "r") as f:
        loaded_label_positions = json.load(f)
    print(f"✅ Loaded label positions for {len(loaded_label_positions)} responses")
else:
    print("🔄 Using newly generated label positions (or will generate if needed)")
    loaded_label_positions = None

# Load steering vectors if file is specified  
if LOAD_STEERING_VECTORS_FILE and os.path.exists(LOAD_STEERING_VECTORS_FILE):
    print(f"📂 Loading steering vectors from {LOAD_STEERING_VECTORS_FILE}")
    loaded_steering_vectors = torch.load(LOAD_STEERING_VECTORS_FILE, map_location='cpu')
    print(f"✅ Loaded steering vectors: {list(loaded_steering_vectors.keys())}")
    
    # Use loaded vectors as the combined_vectors for step 7
    combined_vectors = loaded_steering_vectors
    print(f"🎯 Ready for step 7 with loaded vectors: {list(combined_vectors.keys())}")
else:
    print("🔄 Using newly generated steering vectors (or will generate if needed)")
    # combined_vectors will be set from the previous cell's computation
    
print(f"\n📋 Data loading summary:")
print(f"   Label positions: {'Loaded' if loaded_label_positions else 'Generated/New'}")
print(f"   Steering vectors: {'Loaded' if LOAD_STEERING_VECTORS_FILE and os.path.exists(LOAD_STEERING_VECTORS_FILE) else 'Generated/New'}")

## Step 7: Test Emotional Steering

Now we'll test the emotional steering capabilities by generating responses with different emotional steering settings.

In [ ]:
def test_emotional_steering(model, tokenizer, feature_vectors, test_messages, steering_config):
    """Test emotional steering across different settings"""

    if not feature_vectors:
        print("⚠️  No feature vectors available for testing")
        return []

    results = []
    emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution", "pessimistic-projection"]

    # Also check for any emotional labels we actually found
    available_emotional_labels = [label for label in feature_vectors.keys()
                                 if label != "overall" and any(keyword in label
                                 for keyword in ["thinking", "attribution", "projection", "anxious", "depressive", "negative", "pessimistic"])]

    print(f"🎯 Available emotional labels for testing: {available_emotional_labels}")
    print(f"📊 Total messages to process: {len(test_messages)}")
    print(f"🎛️  Labels per message: {len(available_emotional_labels)}")
    print(f"⚡ Total operations: {len(test_messages) * len(available_emotional_labels) * 2} (pos + neg for each label)")

    for msg_idx, msg in enumerate(tqdm(test_messages, desc="🧪 Processing messages", position=0)):
        message_content = msg["content"]
        
        print(f"\n📝 Message {msg_idx+1}/{len(test_messages)}: {message_content[:50]}{'...' if len(message_content) > 50 else ''}")

        # Generate baseline (no steering)
        try:
            print("  🎯 Generating baseline response...")
            input_ids = tokenizer.encode(message_content, return_tensors="pt")

            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=CONFIG["max_new_tokens"],
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                baseline_output = model.generator.output.save()

            baseline_text = tokenizer.decode(baseline_output[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if baseline_text.startswith(input_text):
                baseline_text = baseline_text[len(input_text):].strip()

            print("  🔍 Analyzing baseline emotional content...")
            baseline_analysis = analyze_emotional_content(baseline_text)
            print(f"  📊 Baseline emotional score: {baseline_analysis.get('total_emotional_score', 0):.1f}%")

            result_entry = {
                "message": message_content,
                "baseline_response": baseline_text,
                "baseline_analysis": baseline_analysis,
                "steered_responses": {}
            }

            # Test steering for each available emotional label
            for label_idx, label in enumerate(tqdm(available_emotional_labels, 
                                                 desc=f"  🎛️  Testing labels", 
                                                 position=1, 
                                                 leave=False)):
                
                print(f"    🔄 [{label_idx+1}/{len(available_emotional_labels)}] Processing: {label}")
                
                # Check if this label has steering config
                model_name = CONFIG["model_name"]
                if model_name in steering_config and label in steering_config[model_name]:

                    # Test positive steering (enhance emotional pattern)
                    with tqdm(total=1, desc=f"      ➕ Positive {label}", position=2, leave=False) as pbar:
                        try:
                            print(f"      ➕ Generating positive steering for {label}...")
                            pos_result = generate_and_analyze_emotional(
                                model, tokenizer, message_content,
                                feature_vectors, steering_config,
                                label, "positive", CONFIG["max_new_tokens"]
                            )

                            result_entry["steered_responses"][f"{label}_positive"] = pos_result
                            pos_score = pos_result.get('emotional_analysis', {}).get('total_emotional_score', 0)
                            print(f"      ✅ Positive {label} completed (score: {pos_score:.1f}%)")
                            pbar.update(1)

                        except Exception as e:
                            print(f"      ❌ Error in positive steering for {label}: {e}")
                            pbar.update(1)

                    # Test negative steering (suppress emotional pattern)
                    with tqdm(total=1, desc=f"      ➖ Negative {label}", position=2, leave=False) as pbar:
                        try:
                            print(f"      ➖ Generating negative steering for {label}...")
                            neg_result = generate_and_analyze_emotional(
                                model, tokenizer, message_content,
                                feature_vectors, steering_config,
                                label, "negative", CONFIG["max_new_tokens"]
                            )

                            result_entry["steered_responses"][f"{label}_negative"] = neg_result
                            neg_score = neg_result.get('emotional_analysis', {}).get('total_emotional_score', 0)
                            print(f"      ✅ Negative {label} completed (score: {neg_score:.1f}%)")
                            pbar.update(1)

                        except Exception as e:
                            print(f"      ❌ Error in negative steering for {label}: {e}")
                            pbar.update(1)

                else:
                    print(f"    ⚠️  No steering config found for {label}")

            results.append(result_entry)
            completed_ops = (msg_idx + 1) * len(available_emotional_labels) * 2
            total_ops = len(test_messages) * len(available_emotional_labels) * 2
            print(f"  ✅ Message {msg_idx+1} completed! ({completed_ops}/{total_ops} total operations)")

        except Exception as e:
            print(f"  ❌ Error processing message {msg_idx+1}: {e}")
            continue

    print(f"\n🎉 All testing completed! Processed {len(results)} messages successfully.")
    return results

# Test on a subset of emotional evaluation messages
test_messages = eval_messages[-5:]  # Use last 5 evaluation messages
print(f"\n🧪 Starting emotional steering tests...")
print(f"📊 Using {len(test_messages)} test messages")

if combined_vectors:
    print("🎯 Feature vectors available - proceeding with tests...")
    
    # Show progress estimation
    available_emotional_labels = [label for label in combined_vectors.keys()
                                 if label != "overall" and any(keyword in label
                                 for keyword in ["thinking", "attribution", "projection", "anxious", "depressive", "negative", "pessimistic"])]
    
    estimated_time = len(test_messages) * len(available_emotional_labels) * 2 * 30  # ~30 seconds per operation
    print(f"⏱️  Estimated time: {estimated_time//60} minutes ({estimated_time} seconds)")
    print("🚀 Starting tests...")
    
    steering_results = test_emotional_steering(
        model, tokenizer, combined_vectors, test_messages, steering_config
    )

    print(f"✅ Completed steering tests for {len(steering_results)} messages")

    # Save results with progress indicator
    print("💾 Saving results...")
    with tqdm(total=1, desc="💾 Saving to file") as pbar:
        with open(f"{CONFIG['results_dir']}/data/steering_results_{CONFIG['timestamp']}.json", "w") as f:
            json.dump(steering_results, f, indent=2)
        pbar.update(1)

    print(f"💾 Saved steering results to steering_results_{CONFIG['timestamp']}.json")
else:
    print("⚠️  No feature vectors available for testing")
    steering_results = []

## Step 8: Analyze and Visualize Results

Analyze the effectiveness of emotional steering and create visualizations.

In [ ]:
def analyze_steering_effectiveness(results):
    """Analyze the effectiveness of emotional steering"""

    if not results:
        print("⚠️  No results to analyze")
        return

    analysis_data = []

    for result in results:
        baseline_scores = result["baseline_analysis"]

        # Analyze each steering condition
        for steering_type, steered_result in result["steered_responses"].items():
            if "emotional_analysis" in steered_result:
                steered_scores = steered_result["emotional_analysis"]

                label, direction = steering_type.split("_")

                analysis_data.append({
                    "message": result["message"][:50] + "...",
                    "label": label,
                    "direction": direction,
                    "baseline_depressive": baseline_scores["depressive_score"],
                    "steered_depressive": steered_scores["depressive_score"],
                    "baseline_anxious": baseline_scores["anxious_score"],
                    "steered_anxious": steered_scores["anxious_score"],
                    "baseline_negative_attribution": baseline_scores["negative_attribution_score"],
                    "steered_negative_attribution": steered_scores["negative_attribution_score"],
                    "baseline_pessimistic": baseline_scores["pessimistic_score"],
                    "steered_pessimistic": steered_scores["pessimistic_score"],
                    "baseline_total": baseline_scores["total_emotional_score"],
                    "steered_total": steered_scores["total_emotional_score"],
                })

    df = pd.DataFrame(analysis_data)

    if len(df) == 0:
        print("⚠️  No valid analysis data found")
        return

    # Calculate effectiveness metrics
    print("📊 Steering Effectiveness Analysis:")
    print("=" * 50)

    # Group by label and direction
    for label in df["label"].unique():
        print(f"\n🎯 {label.replace('-', ' ').title()}:")

        for direction in ["positive", "negative"]:
            subset = df[(df["label"] == label) & (df["direction"] == direction)]

            if len(subset) > 0:
                # Calculate score changes
                score_col = f"steered_{label.replace('-', '_')}"
                baseline_col = f"baseline_{label.replace('-', '_')}"

                if score_col in subset.columns and baseline_col in subset.columns:
                    avg_change = (subset[score_col] - subset[baseline_col]).mean()

                    expected_change = "increase" if direction == "positive" else "decrease"
                    effectiveness = "✅" if (direction == "positive" and avg_change > 0) or (direction == "negative" and avg_change < 0) else "❌"

                    print(f"   {direction.title()} steering: {avg_change:.2f} change (expected {expected_change}) {effectiveness}")

    return df

# Analyze results
if steering_results:
    analysis_df = analyze_steering_effectiveness(steering_results)

    if analysis_df is not None and len(analysis_df) > 0:
        # Save analysis
        analysis_df.to_csv(f"{CONFIG['results_dir']}/data/steering_analysis_{CONFIG['timestamp']}.csv", index=False)
        print(f"\n💾 Saved analysis to CSV")
    else:
        print("⚠️  No analysis data to save")
else:
    print("⚠️  No steering results available for analysis")
    analysis_df = None

## Step 9: Create Visualizations

In [ ]:
def create_steering_visualizations(analysis_df, results_dir):
    """Create visualizations of steering effectiveness"""

    if analysis_df is None or len(analysis_df) == 0:
        print("⚠️  No data available for visualization")
        return

    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Emotional Reasoning Steering Effectiveness', fontsize=16, fontweight='bold')

    emotional_categories = [
        ('depressive', 'Depressive Thinking'),
        ('anxious', 'Anxious Thinking'),
        ('negative_attribution', 'Negative Attribution'),
        ('pessimistic', 'Pessimistic Projection')
    ]

    for idx, (category, title) in enumerate(emotional_categories):
        ax = axes[idx // 2, idx % 2]

        # Filter data for this category
        category_data = analysis_df[analysis_df['label'] == category.replace('_', '-')]

        if len(category_data) == 0:
            ax.text(0.5, 0.5, f'No data for\n{title}',
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title(title)
            continue

        # Prepare data for plotting
        baseline_col = f'baseline_{category}'
        steered_col = f'steered_{category}'

        if baseline_col in category_data.columns and steered_col in category_data.columns:
            pos_data = category_data[category_data['direction'] == 'positive']
            neg_data = category_data[category_data['direction'] == 'negative']

            x_pos = np.arange(len(pos_data))
            width = 0.35

            if len(pos_data) > 0:
                ax.bar(x_pos - width/2, pos_data[baseline_col], width,
                      label='Baseline', alpha=0.7, color='gray')
                ax.bar(x_pos + width/2, pos_data[steered_col], width,
                      label='Positive Steering', alpha=0.7, color='red')

            if len(neg_data) > 0:
                x_neg = np.arange(len(pos_data), len(pos_data) + len(neg_data))
                ax.bar(x_neg - width/2, neg_data[baseline_col], width,
                      alpha=0.7, color='gray')
                ax.bar(x_neg + width/2, neg_data[steered_col], width,
                      label='Negative Steering', alpha=0.7, color='blue')

            ax.set_title(title)
            ax.set_ylabel('Emotional Score (%)')
            ax.legend()
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, f'Data columns missing\nfor {title}',
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title(title)

    plt.tight_layout()
    plt.savefig(f'{results_dir}/figures/emotional_steering_effectiveness_{CONFIG["timestamp"]}.png',
                dpi=300, bbox_inches='tight')
    plt.show()

    # Create summary heatmap
    plt.figure(figsize=(10, 8))

    # Calculate average changes for heatmap
    heatmap_data = []
    labels = []

    for category, title in emotional_categories:
        category_label = category.replace('_', '-')
        pos_data = analysis_df[(analysis_df['label'] == category_label) &
                              (analysis_df['direction'] == 'positive')]
        neg_data = analysis_df[(analysis_df['label'] == category_label) &
                              (analysis_df['direction'] == 'negative')]

        baseline_col = f'baseline_{category}'
        steered_col = f'steered_{category}'

        pos_change = 0
        neg_change = 0

        if len(pos_data) > 0 and baseline_col in pos_data.columns:
            pos_change = (pos_data[steered_col] - pos_data[baseline_col]).mean()

        if len(neg_data) > 0 and baseline_col in neg_data.columns:
            neg_change = (neg_data[steered_col] - neg_data[baseline_col]).mean()

        heatmap_data.append([pos_change, neg_change])
        labels.append(title)

    if heatmap_data:
        sns.heatmap(heatmap_data,
                   xticklabels=['Positive Steering', 'Negative Steering'],
                   yticklabels=labels,
                   annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                   cbar_kws={'label': 'Average Score Change (%)'})

        plt.title('Emotional Steering Effectiveness Heatmap', fontweight='bold', pad=20)
        plt.xlabel('Steering Direction')
        plt.ylabel('Emotional Category')
        plt.tight_layout()
        plt.savefig(f'{results_dir}/figures/emotional_steering_heatmap_{CONFIG["timestamp"]}.png',
                    dpi=300, bbox_inches='tight')
        plt.show()

    print(f"📊 Visualizations saved to {results_dir}/figures/")

# Create visualizations
if analysis_df is not None:
    create_steering_visualizations(analysis_df, CONFIG["results_dir"])
else:
    print("📊 Skipping visualizations - no analysis data available")

## Step 10: Example Usage and Demo

Demonstrate how to use the emotional steering system with specific examples.

In [ ]:
def demo_emotional_steering(model, tokenizer, feature_vectors, steering_config):
    """Demonstrate emotional steering with examples"""

    if not feature_vectors:
        print("⚠️  No feature vectors available for demo")
        return

    demo_messages = [
        "You've been working on a creative project, but it's not turning out as you hoped. How do you feel about your creative abilities?",
        "You have an important presentation tomorrow. What thoughts are going through your mind?",
        "You received some constructive feedback on your work. How do you interpret this feedback?"
    ]

    # Get available emotional labels from our vectors
    available_emotional_labels = [label for label in feature_vectors.keys()
                                 if label != "overall" and any(keyword in label
                                 for keyword in ["thinking", "attribution", "projection", "anxious", "depressive", "negative", "pessimistic"])]

    if not available_emotional_labels:
        print("⚠️  No emotional labels found in feature vectors")
        return

    print("🎭 Emotional Steering Demo")
    print("=" * 50)
    print(f"🎯 Available emotional labels: {available_emotional_labels}")

    for i, message in enumerate(demo_messages[:min(len(demo_messages), len(available_emotional_labels))]):
        label = available_emotional_labels[i % len(available_emotional_labels)]
        model_name = CONFIG["model_name"]

        if model_name not in steering_config or label not in steering_config[model_name]:
            print(f"⚠️  Skipping {label} - steering config not available")
            continue

        print(f"\n📝 Message {i+1}: {message}")
        print(f"🎯 Demonstrating {label.replace('-', ' ').title()} steering")

        try:
            # Baseline response
            input_ids = tokenizer.encode(message, return_tensors="pt")

            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=200,
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                baseline_output = model.generator.output.save()

            baseline_text = tokenizer.decode(baseline_output[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if baseline_text.startswith(input_text):
                baseline_text = baseline_text[len(input_text):].strip()

            print(f"\n🔵 Baseline Response:")
            print(f"   {baseline_text[:300]}...")

            baseline_analysis = analyze_emotional_content(baseline_text)
            print(f"   Emotional Score: {baseline_analysis['total_emotional_score']:.1f}%")

            # Positive steering (enhance emotional pattern)
            pos_result = generate_and_analyze_emotional(
                model, tokenizer, message, feature_vectors, steering_config,
                label, "positive", 200
            )

            print(f"\n🔴 Enhanced {label.replace('-', ' ').title()}:")
            print(f"   {pos_result['response'][:300]}...")
            print(f"   Emotional Score: {pos_result['emotional_analysis']['total_emotional_score']:.1f}%")

            # Negative steering (suppress emotional pattern)
            neg_result = generate_and_analyze_emotional(
                model, tokenizer, message, feature_vectors, steering_config,
                label, "negative", 200
            )

            print(f"\n🟢 Suppressed {label.replace('-', ' ').title()}:")
            print(f"   {neg_result['response'][:300]}...")
            print(f"   Emotional Score: {neg_result['emotional_analysis']['total_emotional_score']:.1f}%")

        except Exception as e:
            print(f"❌ Error in demo for {label}: {e}")
            import traceback
            traceback.print_exc()
            continue

    print(f"\n✅ Demo completed!")

# Run demo
if combined_vectors:
    demo_emotional_steering(model, tokenizer, combined_vectors, steering_config)
else:
    print("⚠️  Demo skipped - no feature vectors available")

## Step 11: Safety and Ethical Considerations

Document important safety considerations and ethical guidelines for this research.

In [ ]:
def generate_safety_report():
    """Generate a safety and ethics report for emotional steering research"""

    report = f"""
# Emotional Reasoning Steering - Safety and Ethics Report

**Generated on:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
**Model:** {CONFIG['model_name']}
**Research Session ID:** {CONFIG['timestamp']}

## Safety Considerations

### 1. Research-Only Use
- This implementation is designed exclusively for research purposes
- Should NOT be deployed in production systems without extensive safety testing
- Requires institutional review board (IRB) approval for human subjects research

### 2. Potential Risks
- **Psychological Harm**: Steering toward negative emotional states could be harmful
- **Misuse**: Could be used to manipulate users or create harmful content
- **Bias Amplification**: May amplify existing biases in training data
- **Unintended Effects**: Steering may have unpredictable side effects

### 3. Required Safeguards
- **Informed Consent**: Users must know when emotional steering is active
- **Monitoring**: Continuous monitoring for harmful outputs
- **Reversibility**: Always provide countermeasures (negative steering)
- **Access Controls**: Restrict access to authorized researchers only
- **Documentation**: Maintain detailed logs of all experiments

## Ethical Guidelines

### 1. Beneficence
- Research should aim to benefit society and advance scientific knowledge
- Potential applications in mental health research and AI safety
- Must not cause unnecessary harm to participants or society

### 2. Autonomy
- Respect user autonomy and decision-making capacity
- Provide clear information about emotional steering effects
- Allow users to opt-out at any time

### 3. Justice
- Ensure fair distribution of research benefits and risks
- Consider impacts on vulnerable populations
- Avoid discriminatory or biased applications

### 4. Non-maleficence
- "Do no harm" - minimize risks to participants and society
- Implement robust safety measures
- Have emergency stop procedures in place

## Recommended Usage Protocols

### 1. Before Starting Research
- Obtain IRB approval for human subjects research
- Develop comprehensive safety protocols
- Train all research staff on ethical considerations
- Establish data security and privacy protections

### 2. During Research
- Monitor all outputs for harmful content
- Maintain detailed experimental logs
- Provide psychological support resources to participants
- Regular safety reviews and protocol updates

### 3. After Research
- Secure deletion of sensitive data
- Debrief participants about the research
- Report findings responsibly to scientific community
- Consider long-term societal implications

## Technical Safety Measures

### 1. Content Filtering
- Implement automated content filtering for harmful outputs
- Human review of all research outputs
- Real-time monitoring of emotional intensity

### 2. Access Controls
- Multi-factor authentication for system access
- Role-based permissions for different user types
- Audit logs of all system interactions

### 3. Data Security
- Encryption of all research data
- Secure storage with limited access
- Regular security audits and updates

## Conclusion

Emotional reasoning steering is a powerful research tool that requires careful ethical consideration and robust safety measures. This research should only be conducted by qualified researchers with appropriate oversight and safeguards in place.

For questions about this research or to report safety concerns, please contact the research team immediately.
"""

    return report

# Generate and save safety report
safety_report = generate_safety_report()

with open(f"{CONFIG['results_dir']}/safety_ethics_report_{CONFIG['timestamp']}.md", "w") as f:
    f.write(safety_report)

print("🛡️ Safety and Ethics Report Generated")
print("=" * 50)
print(safety_report)
print(f"\n💾 Full report saved to: {CONFIG['results_dir']}/safety_ethics_report_{CONFIG['timestamp']}.md")

## Summary and Next Steps

This notebook has provided a complete implementation of emotional reasoning steering for language models. Here's what we accomplished:

### ✅ Completed Tasks:
1. **Extended the COT-steering framework** to include emotional reasoning categories
2. **Generated training data** with emotionally-charged prompts
3. **Trained steering vectors** for depressive, anxious, negative attribution, and pessimistic thinking patterns
4. **Implemented steering functions** to enhance or suppress emotional patterns
5. **Created evaluation metrics** to measure emotional content in responses
6. **Tested the system** with various emotional steering configurations
7. **Generated visualizations** to analyze steering effectiveness
8. **Documented safety and ethical considerations**

### 🔬 Research Applications:
- **Mental Health Research**: Understanding how AI models represent emotional states
- **Bias Detection**: Identifying problematic thinking patterns in AI outputs
- **Therapeutic AI**: Training models to recognize and counter negative thought patterns
- **Content Moderation**: Detecting and filtering emotionally harmful content
- **AI Safety**: Understanding and controlling emotional biases in language models

### ⚠️ Important Reminders:
- This is a **research tool only** - not for production use
- Requires **ethical oversight** and **IRB approval** for human subjects research
- Must include **safety safeguards** and **continuous monitoring**
- Should always provide **positive counterbalancing** capabilities

### 🚀 Next Steps:
1. **Expand training data** with more diverse emotional prompts
2. **Fine-tune steering coefficients** for optimal effectiveness
3. **Implement real-time safety monitoring** systems
4. **Conduct longitudinal studies** on steering effectiveness
5. **Develop therapeutic applications** with proper clinical oversight
6. **Create automated bias detection** systems for production AI

Remember to use this technology responsibly and always prioritize user safety and well-being in your research.

In [ ]:
# Final summary of all generated files
print("📁 Generated Files Summary")
print("=" * 50)

files_generated = [
    f"Data files in {CONFIG['results_dir']}/data/:",
    f"  - emotional_annotations_{CONFIG['timestamp']}.json",
    f"  - emotional_mean_vectors_{CONFIG['model_name'].split('/')[-1].lower()}_{CONFIG['timestamp']}.pt",
    f"  - steering_results_{CONFIG['timestamp']}.json",
    f"  - steering_analysis_{CONFIG['timestamp']}.csv",
    f"",
    f"Visualization files in {CONFIG['results_dir']}/figures/:",
    f"  - emotional_steering_effectiveness_{CONFIG['timestamp']}.png",
    f"  - emotional_steering_heatmap_{CONFIG['timestamp']}.png",
    f"",
    f"Documentation files in {CONFIG['results_dir']}/:",
    f"  - safety_ethics_report_{CONFIG['timestamp']}.md",
]

for file_info in files_generated:
    print(file_info)

print(f"\n🎉 Emotional Reasoning Steering Implementation Complete!")
print(f"📊 Session ID: {CONFIG['timestamp']}")
print(f"🤖 Model: {CONFIG['model_name']}")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
import json

# Define the filename using the timestamp from CONFIG
filename = f"{CONFIG['results_dir']}/data/emotional_responses_{CONFIG['timestamp']}.json"

# Save the emotional responses to a JSON file
with open(filename, "w") as f:
    json.dump(emotional_responses, f, indent=4)

print(f"💾 Saved emotional responses to {filename}")

💾 Saved emotional responses to ./results/data/emotional_responses_20250726_232138.json
